In [1]:
import pandas as pd

# Mostrar todas las columnas sin límite
pd.set_option('display.max_columns', None)

# Cargar el CSV en un DataFrame
df = pd.read_csv("data/processed/final_combined_2023_2024.csv")

# Mostrar primeras filas
display(df.head())

# Imprimir nombres de columnas
print(df.columns.tolist())


/var/folders/z2/yk2bcpgn17zcc7cpjntmvq7w0000gn/T/ipykernel_23092/620114380.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/processed/final_combined_2023_2024.csv")


,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes,tavg,tmin,tmax,prcp,event(y/n),name,attendance
0,2023-01-01,Domingo,1,Internacional,8001,Abrera,8003,Alella,0,11.0,8.6,15.9,0.0,n,NaN,NaN
1,2023-01-01,Domingo,1,Internacional,8204,Sant Climent de Llobregat,8073,Cornellà de Llobregat,0,11.0,8.6,15.9,0.0,n,NaN,NaN
2,2023-01-01,Domingo,1,Residente,8204,Sant Climent de Llobregat,8073,Cornellà de Llobregat,0,11.0,8.6,15.9,0.0,n,NaN,NaN
3,2023-01-01,Domingo,1,Regional,8204,Sant Climent de Llobregat,8074,Cubelles,0,11.0,8.6,15.9,0.0,n,NaN,NaN
4,2023-01-01,Domingo,1,Regional,8204,Sant Climent de Llobregat,8076,Esparreguera,0,11.0,8.6,15.9,0.0,n,NaN,NaN


['day', 'day_of_week', 'month', 'origen', 'municipio_origen', 'municipio_origen_name', 'municipio_destino', 'municipio_destino_name', 'viajes', 'tavg', 'tmin', 'tmax', 'prcp', 'event(y/n)', 'name', 'attendance']


In [2]:
import pandas as pd

# === 1) Cargar datos ===
df = pd.read_csv("data/processed/final_combined_2023_2024.csv")

# Crear columnas más cortas para comodidad
A = df["municipio_origen_name"]
B = df["municipio_destino_name"]

# === 2) PAREJAS A -> B ===
pairs_AB = set(zip(A, B))
print("N° de enlaces únicos A -> B:", len(pairs_AB))

# === 3) PAREJAS B -> A (invertidas) ===
pairs_BA = set(zip(B, A))
print("N° de enlaces únicos B -> A:", len(pairs_BA))

# === 4) PAREJAS IGNORANDO DIRECCIÓN ===
# Cada par se ordena alfabéticamente (A,B) == (B,A)
undirected_pairs = set(tuple(sorted(pair)) for pair in pairs_AB)
print("N° de enlaces únicos sin dirección (A,B == B,A):", len(undirected_pairs))

# === 5) Ver si los dos sets dirigidos generan el mismo conjunto sin dirección ===
# Crear version sin dirección para pairs_BA
undirected_pairs_BA = set(tuple(sorted(pair)) for pair in pairs_BA)

# Comparación
sets_equal = undirected_pairs == undirected_pairs_BA
print("¿Los conjuntos son iguales ignorando dirección?:", sets_equal)

# === 6) Ver diferencias ===
only_AB = undirected_pairs - undirected_pairs_BA
only_BA = undirected_pairs_BA - undirected_pairs

print("Parejas solo en A->B (sin su inversa):", len(only_AB))
print("Parejas solo en B->A (sin su inversa):", len(only_BA))

if len(only_AB) > 0:
    print("Ejemplos de parejas solo en A->B:", list(only_AB)[:10])

if len(only_BA) > 0:
    print("Ejemplos de parejas solo en B->A:", list(only_BA)[:10])


/var/folders/z2/yk2bcpgn17zcc7cpjntmvq7w0000gn/T/ipykernel_23092/1444655360.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/processed/final_combined_2023_2024.csv")


N° de enlaces únicos A -> B: 88352
N° de enlaces únicos B -> A: 88352
N° de enlaces únicos sin dirección (A,B == B,A): 45029
¿Los conjuntos son iguales ignorando dirección?: True
Parejas solo en A->B (sin su inversa): 0
Parejas solo en B->A (sin su inversa): 0


In [3]:
import pandas as pd
import numpy as np

# === 1. Cargar CSV de municipios con lat/lon ===
df_coords = pd.read_csv("data/processed/municipios_with_lat_alt.csv")

# Mira qué columnas vienen exactamente
print(df_coords.columns)

# Si las columnas tienen otros nombres, ajustamos abajo.


Index(['municipio', 'lat', 'alt', 'lon'], dtype='object')


In [4]:
# === 2. Coordenadas de Barcelona (centro aproximado) ===
lat_bcn = 41.3851
lon_bcn = 2.1734

# === 3. Función Haversine ===
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radio de la Tierra en km

    # Convertir a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

# === 4. Calcular distancia de Barcelona a cada municipio ===
df_coords["dist_km_to_barcelona"] = df_coords.apply(
    lambda row: haversine(lat_bcn, lon_bcn, row["lat"], row["lon"]),
    axis=1
)

# === 5. Mostrar ordenado por distancia
df_coords_sorted = df_coords.sort_values("dist_km_to_barcelona")
df_coords_sorted.head(15)


,municipio,lat,alt,lon,dist_km_to_barcelona
18,Barcelona,41.382580,30.442980,2.177073,0.415221
86,"Garriga, La",41.390782,43.643982,2.165454,0.915760
101,"Llacuna, La",41.402663,9.904580,2.197194,2.784429
188,Sant Adrià de Besòs,41.430483,18.403013,2.218278,6.282826
98,"Hospitalet de Llobregat, L'",41.359860,16.908049,2.099793,6.752912
75,Esplugues de Llobregat,41.377680,98.987358,2.089972,7.009317
238,Santa Coloma de Gramenet,41.451563,38.885090,2.208337,7.943771
215,Sant Just Desvern,41.381583,108.159378,2.075068,8.213170
166,"Prat de Llobregat, El",41.330792,10.109638,2.093019,9.026348
71,Cornellà de Llobregat,41.355724,36.739948,2.070622,9.177407


In [6]:
municipio_col = "municipio"

limits = {
    "under_10km": 10,
    "under_20km": 20,
    "under_30km": 30,
    "under_50km": 50
}

municipios_in_range = {}

for key, max_dist in limits.items():
    subset = df_coords[df_coords["dist_km_to_barcelona"] < max_dist]
    municipios_in_range[key] = list(subset[municipio_col])

    print(f"\nMunicipios a menos de {max_dist} km:")
    print(municipios_in_range[key])
    print("Total:", len(municipios_in_range[key]))



Municipios a menos de 10 km:
['Badalona', 'Barcelona', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Garriga, La', "Hospitalet de Llobregat, L'", 'Llacuna, La', 'Prat de Llobregat, El', 'Sant Adrià de Besòs', 'Sant Joan Despí', 'Sant Just Desvern', 'Santa Coloma de Gramenet']
Total: 12

Municipios a menos de 20 km:
['Alella', 'Badalona', 'Barcelona', 'Bruc, El', 'Castellbisbal', 'Castelldefels', 'Cervelló', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Garriga, La', 'Gavà', "Hospitalet de Llobregat, L'", 'Llacuna, La', 'Llagosta, La', 'Martorelles', 'Masnou, El', 'Molins de Rei', 'Mollet del Vallès', 'Montcada i Reixac', 'Montgat', 'Montmeló', 'Montornès del Vallès', 'Pallejà', 'Papiol, El', 'Polinyà', 'Prat de Llobregat, El', 'Premià de Mar', 'Ripollet', 'Rubí', 'Sabadell', 'Sant Adrià de Besòs', 'Sant Andreu de la Barca', 'Sant Boi de Llobregat', 'Sant Climent de Llobregat', 'Sant Cugat del Vallès', 'Sant Fost de Campsentelles', 'Sant Feliu de Llobregat', 'Sant Joan Desp

In [8]:
municipios_modelo = [
    # ---------------------------
    # AMB — BARCELONÈS
    # ---------------------------
    'Barcelona',                         # AMB
    'Badalona',                          # AMB
    'Santa Coloma de Gramenet',          # AMB
    'Sant Adrià de Besòs',               # AMB
    "Hospitalet de Llobregat, L'",       # AMB

    # ---------------------------
    # AMB — BAIX LLOBREGAT
    # ---------------------------
    'Castelldefels',                     # AMB
    'Cornellà de Llobregat',             # AMB
    'Esplugues de Llobregat',            # AMB
    'Gavà',                              # AMB
    'Molins de Rei',                     # AMB
    'Pallejà',                           # AMB
    'Papiol, El',                        # AMB
    'Sant Andreu de la Barca',           # AMB
    'Sant Boi de Llobregat',             # AMB
    'Sant Climent de Llobregat',         # AMB
    'Sant Feliu de Llobregat',           # AMB
    'Sant Joan Despí',                   # AMB
    'Sant Just Desvern',                 # AMB
    'Santa Coloma de Cervelló',          # AMB
    'Torrelles de Llobregat',            # AMB
    'Palma de Cervelló, La',             # AMB
    'Cervelló',                          # AMB
    'Vallirana',                         # AMB
    'Viladecans',                        # AMB
    'Prat de Llobregat, El',             # AMB

    # ---------------------------
    # AMB — VALLÈS OCCIDENTAL
    # ---------------------------
    'Badia del Vallès',                  # AMB
    'Barberà del Vallès',                # AMB
    'Cerdanyola del Vallès',             # AMB
    'Montcada i Reixac',                 # AMB
    'Ripollet',                          # AMB
    'Rubí',                              # AMB
    'Sant Cugat del Vallès',             # AMB
    'Sant Quirze del Vallès',            # AMB
    'Santa Perpètua de Mogoda',          # AMB

    # ---------------------------
    # AMB — MARESME
    # ---------------------------
    'Montgat',                           # AMB
    'Tiana',                             # AMB

    # -----------------------------------------------------
    # 🔵 MUNICIPIOS RELEVANTES *NO AMB* (añadidos mínimos)
    # -----------------------------------------------------

    # --- VALLÈS OCCIDENTAL (núcleos muy conectados con BCN) ---
    'Sabadell',                          # NO AMB — importante
    'Terrassa',                          # NO AMB — importante
    'Castellar del Vallès',              # NO AMB — relevante
    'Sentmenat',                         # NO AMB — relevante
    'Palau-solità i Plegamans',          # NO AMB — relevante
    'Polinyà',                           # NO AMB — relevante

    # --- PRIMERA CORONA VALLÈS ORIENTAL ---
    'Montmeló',                          # NO AMB
    'Montornès del Vallès',              # NO AMB
    'Martorelles',                       # NO AMB
    'Santa Maria de Martorelles',        # NO AMB
    'Canovelles',                        # NO AMB
    'Franqueses del Vallès, Les',        # NO AMB
    'Granollers',                        # NO AMB
    'Parets del Vallès',                 # NO AMB
    'Llagosta, La',                      # NO AMB
    'Vilanova del Vallès',               # NO AMB
    'Vallromanes',                       # NO AMB

    # --- MARESME CERCANO ---
    'Masnou, El',                        # NO AMB
    'Alella',                            # NO AMB
    'Teià',                              # NO AMB
    'Premià de Mar',                     # NO AMB
    'Premià de Dalt',                    # NO AMB
    'Vilassar de Mar',                   # NO AMB
    'Vilassar de Dalt',                  # NO AMB
    'Cabrera de Mar',                    # NO AMB
    'Cabrils',                           # NO AMB
    'Mataró',                            # NO AMB — muy relevante
    'Argentona',                         # NO AMB — relevante
]


In [9]:
# 1. Número total de municipios en tu lista
print("Número de municipios en municipios_modelo:", len(municipios_modelo))

# 2. Extraer todos los nombres que aparecen en el dataset de enlaces (origen + destino)
municipios_en_enlaces = set(df["municipio_origen_name"].unique()) | set(df["municipio_destino_name"].unique())

# 3. Convertimos tu lista en set para facilitar comparaciones
municipios_modelo_set = set(municipios_modelo)

# 4. Municipios del modelo que SÍ aparecen en el dataset de enlaces
presentes = municipios_modelo_set & municipios_en_enlaces

# 5. Municipios del modelo que NO aparecen en el dataset de enlaces
ausentes = municipios_modelo_set - municipios_en_enlaces

print("\nMunicipios del modelo que SÍ aparecen en los datos:", len(presentes))
print(sorted(list(presentes)))

print("\nMunicipios del modelo que NO aparecen en los datos:", len(ausentes))
print(sorted(list(ausentes)))


Número de municipios en municipios_modelo: 64

Municipios del modelo que SÍ aparecen en los datos: 64
['Alella', 'Argentona', 'Badalona', 'Badia del Vallès', 'Barberà del Vallès', 'Barcelona', 'Cabrera de Mar', 'Cabrils', 'Canovelles', 'Castellar del Vallès', 'Castelldefels', 'Cerdanyola del Vallès', 'Cervelló', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Franqueses del Vallès, Les', 'Gavà', 'Granollers', "Hospitalet de Llobregat, L'", 'Llagosta, La', 'Martorelles', 'Masnou, El', 'Mataró', 'Molins de Rei', 'Montcada i Reixac', 'Montgat', 'Montmeló', 'Montornès del Vallès', 'Palau-solità i Plegamans', 'Pallejà', 'Palma de Cervelló, La', 'Papiol, El', 'Parets del Vallès', 'Polinyà', 'Prat de Llobregat, El', 'Premià de Dalt', 'Premià de Mar', 'Ripollet', 'Rubí', 'Sabadell', 'Sant Adrià de Besòs', 'Sant Andreu de la Barca', 'Sant Boi de Llobregat', 'Sant Climent de Llobregat', 'Sant Cugat del Vallès', 'Sant Feliu de Llobregat', 'Sant Joan Despí', 'Sant Just Desvern', 'Sant Quirze d

In [10]:
# Convertimos la lista a set para filtrado rápido
municipios_set = set(municipios_modelo)

# Número de filas antes del filtrado
rows_before = len(df)

# Filtrar filas donde ORIGEN y DESTINO están en la lista
df_reduced = df[
    df["municipio_origen_name"].isin(municipios_set) &
    df["municipio_destino_name"].isin(municipios_set)
].copy()

# Número de filas después del filtrado
rows_after = len(df_reduced)

print("Filas antes del filtrado:", rows_before)
print("Filas después del filtrado:", rows_after)
print("Filas eliminadas:", rows_before - rows_after)


Filas antes del filtrado: 48598137
Filas después del filtrado: 7584465
Filas eliminadas: 41013672


In [17]:
# Número de filas antes (en el df reducido)
rows_before = len(df_reduced)

# Filtrar solo los enlaces donde el destino es Barcelona
df_reduced = df_reduced[df_reduced["municipio_destino_name"] == "Barcelona"].copy()

# Número de filas después
rows_after = len(df_reduced)

print("Filas en df_reduced antes:", rows_before)
print("Filas en df_reduced_to_barcelona:", rows_after)
print("Filas eliminadas:", rows_before - rows_after)


Filas en df_reduced antes: 7584465
Filas en df_reduced_to_barcelona: 183449
Filas eliminadas: 7401016


In [18]:
# Municipios presentes como ORIGEN
origen_unicos = sorted(df_reduced["municipio_origen_name"].unique())

# Municipios presentes como DESTINO
destino_unicos = sorted(df_reduced["municipio_destino_name"].unique())

print("Municipios presentes como ORIGEN:")
print(origen_unicos)
print("\nMunicipios presentes como DESTINO:")
print(destino_unicos)


Municipios presentes como ORIGEN:
['Alella', 'Argentona', 'Badalona', 'Badia del Vallès', 'Barberà del Vallès', 'Cabrera de Mar', 'Cabrils', 'Canovelles', 'Castellar del Vallès', 'Castelldefels', 'Cerdanyola del Vallès', 'Cervelló', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Franqueses del Vallès, Les', 'Gavà', 'Granollers', "Hospitalet de Llobregat, L'", 'Llagosta, La', 'Martorelles', 'Masnou, El', 'Mataró', 'Molins de Rei', 'Montcada i Reixac', 'Montgat', 'Montmeló', 'Montornès del Vallès', 'Palau-solità i Plegamans', 'Pallejà', 'Palma de Cervelló, La', 'Papiol, El', 'Parets del Vallès', 'Polinyà', 'Prat de Llobregat, El', 'Premià de Dalt', 'Premià de Mar', 'Ripollet', 'Rubí', 'Sabadell', 'Sant Adrià de Besòs', 'Sant Andreu de la Barca', 'Sant Boi de Llobregat', 'Sant Climent de Llobregat', 'Sant Cugat del Vallès', 'Sant Feliu de Llobregat', 'Sant Joan Despí', 'Sant Just Desvern', 'Sant Quirze del Vallès', 'Santa Coloma de Cervelló', 'Santa Coloma de Gramenet', 'Santa Maria 

In [19]:
municipios_presentes = sorted(
    set(origen_unicos).union(destino_unicos)
)

print("Municipios presentes (ORIGEN ∪ DESTINO):")
print(municipios_presentes)
print("\nTotal municipios presentes:", len(municipios_presentes))


Municipios presentes (ORIGEN ∪ DESTINO):
['Alella', 'Argentona', 'Badalona', 'Badia del Vallès', 'Barberà del Vallès', 'Barcelona', 'Cabrera de Mar', 'Cabrils', 'Canovelles', 'Castellar del Vallès', 'Castelldefels', 'Cerdanyola del Vallès', 'Cervelló', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Franqueses del Vallès, Les', 'Gavà', 'Granollers', "Hospitalet de Llobregat, L'", 'Llagosta, La', 'Martorelles', 'Masnou, El', 'Mataró', 'Molins de Rei', 'Montcada i Reixac', 'Montgat', 'Montmeló', 'Montornès del Vallès', 'Palau-solità i Plegamans', 'Pallejà', 'Palma de Cervelló, La', 'Papiol, El', 'Parets del Vallès', 'Polinyà', 'Prat de Llobregat, El', 'Premià de Dalt', 'Premià de Mar', 'Ripollet', 'Rubí', 'Sabadell', 'Sant Adrià de Besòs', 'Sant Andreu de la Barca', 'Sant Boi de Llobregat', 'Sant Climent de Llobregat', 'Sant Cugat del Vallès', 'Sant Feliu de Llobregat', 'Sant Joan Despí', 'Sant Just Desvern', 'Sant Quirze del Vallès', 'Santa Coloma de Cervelló', 'Santa Coloma de Gram

In [20]:
faltan = sorted(list(set(municipios_modelo) - set(municipios_presentes)))

print("Municipios de municipios_modelo que NO aparecen en df_reduced:")
print(faltan)


Municipios de municipios_modelo que NO aparecen en df_reduced:
[]


In [21]:
sobra = sorted(list(set(municipios_presentes) - set(municipios_modelo)))

print("Municipios que aparecen en df_reduced pero NO están en municipios_modelo:")
print(sobra)


Municipios que aparecen en df_reduced pero NO están en municipios_modelo:
[]


In [22]:
print(municipios_presentes)


['Alella', 'Argentona', 'Badalona', 'Badia del Vallès', 'Barberà del Vallès', 'Barcelona', 'Cabrera de Mar', 'Cabrils', 'Canovelles', 'Castellar del Vallès', 'Castelldefels', 'Cerdanyola del Vallès', 'Cervelló', 'Cornellà de Llobregat', 'Esplugues de Llobregat', 'Franqueses del Vallès, Les', 'Gavà', 'Granollers', "Hospitalet de Llobregat, L'", 'Llagosta, La', 'Martorelles', 'Masnou, El', 'Mataró', 'Molins de Rei', 'Montcada i Reixac', 'Montgat', 'Montmeló', 'Montornès del Vallès', 'Palau-solità i Plegamans', 'Pallejà', 'Palma de Cervelló, La', 'Papiol, El', 'Parets del Vallès', 'Polinyà', 'Prat de Llobregat, El', 'Premià de Dalt', 'Premià de Mar', 'Ripollet', 'Rubí', 'Sabadell', 'Sant Adrià de Besòs', 'Sant Andreu de la Barca', 'Sant Boi de Llobregat', 'Sant Climent de Llobregat', 'Sant Cugat del Vallès', 'Sant Feliu de Llobregat', 'Sant Joan Despí', 'Sant Just Desvern', 'Sant Quirze del Vallès', 'Santa Coloma de Cervelló', 'Santa Coloma de Gramenet', 'Santa Maria de Martorelles', 'San

In [23]:
print("Columnas del DataFrame:")
for col in df_reduced.columns:
    print(col)


Columnas del DataFrame:
day
day_of_week
month
origen
municipio_origen
municipio_origen_name
municipio_destino
municipio_destino_name
viajes
tavg
tmin
tmax
prcp
event(y/n)
name
attendance


In [24]:
# ================================================
#   INSPECCIÓN COMPLETA DEL DATAFRAME REDUCIDO
# ================================================

print("===== COLUMNAS =====")
print(df_reduced.columns.tolist())
print("\n")

print("===== TIPOS DE DATOS =====")
print(df_reduced.dtypes)
print("\n")

print("===== 5 FILAS EJEMPLO =====")
print(df_reduced.head(5))
print("\n")

print("===== MUNICIPIOS DESTINO (DEBERÍA SER SOLO 'Barcelona') =====")
print(df_reduced["municipio_destino_name"].unique())
print("\n")

print("===== MUNICIPIOS ORIGEN FINALES =====")
municipios_origen_final = sorted(df_reduced["municipio_origen_name"].unique())
print(municipios_origen_final)
print("\nTotal municipios origen:", len(municipios_origen_final))
print("\n")

# Esto me lo pasas y hago el feature engineering completo
print("===== LISTO. PÁSAMELA SALIDA COMPLETA =====")


===== COLUMNAS =====
['day', 'day_of_week', 'month', 'origen', 'municipio_origen', 'municipio_origen_name', 'municipio_destino', 'municipio_destino_name', 'viajes', 'tavg', 'tmin', 'tmax', 'prcp', 'event(y/n)', 'name', 'attendance']


===== TIPOS DE DATOS =====
day                        object
day_of_week                object
month                       int64
origen                     object
municipio_origen            int64
municipio_origen_name      object
municipio_destino           int64
municipio_destino_name     object
viajes                      int64
tavg                      float64
tmin                      float64
tmax                      float64
prcp                      float64
event(y/n)                 object
name                       object
attendance                float64
dtype: object


===== 5 FILAS EJEMPLO =====
            day day_of_week  month         origen  municipio_origen  \
43   2023-01-01     Domingo      1       Regional              8204   
44   202

In [25]:
df_reduced.head()

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes,tavg,tmin,tmax,prcp,event(y/n),name,attendance
43,2023-01-01,Domingo,1,Regional,8204,Sant Climent de Llobregat,8019,Barcelona,115,11.0,8.6,15.9,0.0,n,NaN,NaN
44,2023-01-01,Domingo,1,Internacional,8204,Sant Climent de Llobregat,8019,Barcelona,25,11.0,8.6,15.9,0.0,n,NaN,NaN
45,2023-01-01,Domingo,1,Residente,8204,Sant Climent de Llobregat,8019,Barcelona,115,11.0,8.6,15.9,0.0,n,NaN,NaN
46,2023-01-01,Domingo,1,Nacional,8204,Sant Climent de Llobregat,8019,Barcelona,0,11.0,8.6,15.9,0.0,n,NaN,NaN
355,2023-01-01,Domingo,1,Nacional,8205,Sant Cugat del Vallès,8019,Barcelona,874,11.0,8.6,15.9,0.0,n,NaN,NaN


In [36]:
events = [
    # 2023 TRADE FAIRS & CONGRESSES
    {"name": "Integrated Systems Europe 2023", "start_date": "2023-01-31", "end_date": "2023-02-03", "attendance": 58107, "category": "trade_fair"},
    {"name": "Mobile World Congress 2023", "start_date": "2023-02-27", "end_date": "2023-03-02", "attendance": 88500, "category": "trade_fair"},
    {"name": "Seafood Expo Global 2023", "start_date": "2023-04-25", "end_date": "2023-04-27", "attendance": 33000, "category": "trade_fair"},
    {"name": "Smart City Expo World Congress 2023", "start_date": "2023-11-07", "end_date": "2023-11-09", "attendance": 25300, "category": "trade_fair"},

    # 2023 FESTIVALS
    {"name": "Primavera Sound Barcelona 2023", "start_date": "2023-06-01", "end_date": "2023-06-03", "attendance": 253000, "category": "festival"},
    {"name": "Sónar Barcelona 2023", "start_date": "2023-06-15", "end_date": "2023-06-17", "attendance": 120000, "category": "festival"},
    {"name": "Festival Cruïlla Barcelona 2023", "start_date": "2023-07-05", "end_date": "2023-07-08", "attendance": 76000, "category": "festival"},

    # 2023 CITY FESTIVALS & MAJOR HOLIDAYS
    {"name": "La Mercè 2023 (Festes de Barcelona)", "start_date": "2023-09-22", "end_date": "2023-09-25", "attendance": 1000000, "category": "city_festival"},
    {"name": "Diada de Sant Jordi 2023", "start_date": "2023-04-23", "end_date": "2023-04-23", "attendance": 1000000, "category": "city_festival"},
    {"name": "Verbena de Sant Joan 2023", "start_date": "2023-06-23", "end_date": "2023-06-23", "attendance": 60000, "category": "city_festival"},
    {"name": "Festa Major de Gràcia 2023", "start_date": "2023-08-15", "end_date": "2023-08-21", "attendance": 2000000, "category": "city_festival"},
    {"name": "Festa Major de Sants 2023", "start_date": "2023-08-19", "end_date": "2023-08-27", "attendance": 300000, "category": "city_festival"},
    {"name": "Cabalgata de Reyes 2023 (Barcelona)", "start_date": "2023-01-05", "end_date": "2023-01-05", "attendance": 500000, "category": "city_festival"},

    # 2023 BIG STADIUM CONCERTS
    {"name": "Bruce Springsteen - Estadi Olímpic Lluís Companys (28 Apr 2023)", "start_date": "2023-04-28", "end_date": "2023-04-28", "attendance": 55000, "category": "concert"},
    {"name": "Bruce Springsteen - Estadi Olímpic Lluís Companys (30 Apr 2023)", "start_date": "2023-04-30", "end_date": "2023-04-30", "attendance": 55000, "category": "concert"},
    {"name": "Coldplay - Estadi Olímpic Lluís Companys (24 May 2023)", "start_date": "2023-05-24", "end_date": "2023-05-24", "attendance": 56000, "category": "concert"},
    {"name": "Coldplay - Estadi Olímpic Lluís Companys (25 May 2023)", "start_date": "2023-05-25", "end_date": "2023-05-25", "attendance": 56000, "category": "concert"},
    {"name": "Coldplay - Estadi Olímpic Lluís Companys (27 May 2023)", "start_date": "2023-05-27", "end_date": "2023-05-27", "attendance": 56000, "category": "concert"},
    {"name": "Coldplay - Estadi Olímpic Lluís Companys (28 May 2023)", "start_date": "2023-05-28", "end_date": "2023-05-28", "attendance": 56000, "category": "concert"},
    {"name": "Harry Styles - Estadi Olímpic Lluís Companys (12 Jul 2023)", "start_date": "2023-07-12", "end_date": "2023-07-12", "attendance": 55000, "category": "concert"},
    {"name": "The Weeknd - Estadi Olímpic Lluís Companys (20 Jul 2023)", "start_date": "2023-07-20", "end_date": "2023-07-20", "attendance": 54000, "category": "concert"},

    # 2024 TRADE FAIRS & CONGRESSES
    {"name": "Integrated Systems Europe 2024", "start_date": "2024-01-30", "end_date": "2024-02-02", "attendance": 73891, "category": "trade_fair"},
    {"name": "Mobile World Congress 2024", "start_date": "2024-02-26", "end_date": "2024-02-29", "attendance": 101000, "category": "trade_fair"},
    {"name": "Alimentaria & Hostelco 2024", "start_date": "2024-03-18", "end_date": "2024-03-21", "attendance": 100000, "category": "trade_fair"},
    {"name": "Seafood Expo Global 2024", "start_date": "2024-04-23", "end_date": "2024-04-25", "attendance": 35000, "category": "trade_fair"},
    {"name": "Smart City Expo World Congress 2024", "start_date": "2024-11-05", "end_date": "2024-11-07", "attendance": 25700, "category": "trade_fair"},

    # 2024 FESTIVALS
    {"name": "Primavera Sound Barcelona 2024", "start_date": "2024-05-30", "end_date": "2024-06-01", "attendance": 268000, "category": "festival"},
    {"name": "Sónar Barcelona 2024", "start_date": "2024-06-13", "end_date": "2024-06-15", "attendance": 154000, "category": "festival"},
    {"name": "Festival Cruïlla Barcelona 2024", "start_date": "2024-07-10", "end_date": "2024-07-13", "attendance": 76000, "category": "festival"},

    # 2024 CITY FESTIVALS & MAJOR HOLIDAYS
    {"name": "La Mercè 2024 (Festes de Barcelona)", "start_date": "2024-09-20", "end_date": "2024-09-24", "attendance": 1000000, "category": "city_festival"},
    {"name": "Diada de Sant Jordi 2024", "start_date": "2024-04-23", "end_date": "2024-04-23", "attendance": 1000000, "category": "city_festival"},
    {"name": "Verbena de Sant Joan 2024", "start_date": "2024-06-23", "end_date": "2024-06-23", "attendance": 60000, "category": "city_festival"},
    {"name": "Festa Major de Gràcia 2024", "start_date": "2024-08-15", "end_date": "2024-08-21", "attendance": 2000000, "category": "city_festival"},
    {"name": "Festa Major de Sants 2024", "start_date": "2024-08-24", "end_date": "2024-09-01", "attendance": 300000, "category": "city_festival"},
    {"name": "Cabalgata de Reyes 2024 (Barcelona)", "start_date": "2024-01-05", "end_date": "2024-01-05", "attendance": 500000, "category": "city_festival"},

    # ------------------------------
    # NEW EVENTS (SEMANTICALLY UNIQUE)
    # ------------------------------

    # Concerts / Shows
    {"name": "Brucee Springsteen (Estadi Olimpic)", "start_date": "2024-06-20", "end_date": "2024-06-20", "attendance": 58000.0, "category": "concert"},
    {"name": "Brucee Springsteen (Estadi Olimpic)", "start_date": "2024-06-22", "end_date": "2024-06-22", "attendance": 58000.0, "category": "concert"},
    {"name": "Estopa (Estadi Olimpic)", "start_date": "2024-07-10", "end_date": "2024-07-10", "attendance": 55000.0, "category": "concert"},
    {"name": "Rammstein (Estadi Olimpic)", "start_date": "2024-06-11", "end_date": "2024-06-11", "attendance": 60000.0, "category": "concert"},

    # Motorsports
    {"name": "Gran Premio F1", "start_date": "2023-06-04", "end_date": "2023-06-04", "attendance": 284066.0, "category": "motorsport"},
    {"name": "Gran Premio MotoGP", "start_date": "2023-09-01", "end_date": "2023-09-03", "attendance": 83000.0, "category": "motorsport"},
    {"name": "Gran Premio MotoGP", "start_date": "2024-05-24", "end_date": "2024-05-26", "attendance": 83000.0, "category": "motorsport"},

    # Football matches
    {"name": "Barça vs Real Madrid (Camp Nou)", "start_date": "2023-03-19", "end_date": "2023-03-19", "attendance": 95745.0, "category": "football"},
    {"name": "Kings League Final Four (Camp Nou)", "start_date": "2023-03-26", "end_date": "2023-03-26", "attendance": 92522.0, "category": "other_sport"},
    {"name": "Liga Barça vs Atlètic", "start_date": "2023-03-12", "end_date": "2023-03-12", "attendance": 88000.0, "category": "football"},
    {"name": "Liga Barça vs Espanyol", "start_date": "2024-11-03", "end_date": "2024-11-03", "attendance": 48443.0, "category": "football"},
    {"name": "Liga Barça vs Real Madrid", "start_date": "2024-10-28", "end_date": "2024-10-28", "attendance": 50112.0, "category": "football"},
    {"name": "Liga Barça vs Real Sociedad", "start_date": "2023-05-20", "end_date": "2023-05-20", "attendance": 88000.0, "category": "football"},
    {"name": "Liga Barça vs Sevilla", "start_date": "2024-10-20", "end_date": "2024-10-20", "attendance": 47848.0, "category": "football"},
    {"name": "Liga Barça vs Sevilla (Camp Nou)", "start_date": "2023-02-05", "end_date": "2023-02-05", "attendance": 89000.0, "category": "football"},

    {"name": "UCWL Semifinal Barça Femení vs Chelsea (Camp Nou)", "start_date": "2023-04-27", "end_date": "2023-04-27", "attendance": 72262.0, "category": "other_sport"},

    {"name": "UEFA Barça vs Antwerp (Estadi Olimpic)", "start_date": "2023-09-19", "end_date": "2023-09-19", "attendance": 40989.0, "category": "football"},
    {"name": "UEFA Barça vs Napoles", "start_date": "2024-03-12", "end_date": "2024-03-12", "attendance": 50301.0, "category": "football"},
    {"name": "UEFA Barça vs PSG", "start_date": "2024-04-16", "end_date": "2024-04-16", "attendance": 50309.0, "category": "football"},
    {"name": "UEFA Barça vs Porto (Estadi Olimpic)", "start_date": "2023-11-28", "end_date": "2023-11-28", "attendance": 43533.0, "category": "football"},
    {"name": "UEFA Barça vs Shaktar (Estadi Olimpic)", "start_date": "2023-10-25", "end_date": "2023-10-25", "attendance": 41.409, "category": "football"},
]


In [37]:
import pandas as pd

def build_event_day_df(events):
    rows = []

    for ev in events:
        start = pd.to_datetime(ev["start_date"])
        end = pd.to_datetime(ev["end_date"])

        # Expand date range day-by-day
        days = pd.date_range(start, end, freq="D")

        for d in days:
            rows.append({
                "day": d,
                "name": ev["name"],
                "category": ev["category"],
                "attendance": ev["attendance"]
            })

    df = pd.DataFrame(rows)

    # Group by day and merge
    df_grouped = df.groupby("day").agg({
        "attendance": "sum",
        "name": lambda x: " + ".join(sorted(set(x))),
        "category": lambda x: " + ".join(sorted(set(x)))  # multi-category
    }).reset_index()

    return df_grouped


# Build the event-day dataframe
df_events_daily = build_event_day_df(events)

df_events_daily.head()


,day,attendance,name,category
0,2023-01-05,500000.0,Cabalgata de Reyes 2023 (Barcelona),city_festival
1,2023-01-31,58107.0,Integrated Systems Europe 2023,trade_fair
2,2023-02-01,58107.0,Integrated Systems Europe 2023,trade_fair
3,2023-02-02,58107.0,Integrated Systems Europe 2023,trade_fair
4,2023-02-03,58107.0,Integrated Systems Europe 2023,trade_fair


In [41]:
import pandas as pd
import numpy as np

# ======================================================
# 0. PREPARAR EVENTOS DIARIOS (df_events_daily ya creado)
# ======================================================

df_events_daily['date'] = pd.to_datetime(df_events_daily['date'])

df_events_daily = df_events_daily.rename(columns={
    "event_names": "event_names",
    "event_category": "event_category",
    "event_attendance": "event_attendance"
})

# ======================================================
# 1. COPIA SEGURA DEL DF REDUCIDO
# ======================================================

df_fe = df_reduced.copy()

df_fe["date"] = pd.to_datetime(df_fe["day"], errors="coerce")
df_fe["month"] = df_fe["month"].astype(int)

# ======================================================
# 2. MERGE DE EVENTOS NUEVOS (EVENTOS EXTERNOS CORREGIDOS)
# ======================================================

df_fe = df_fe.merge(df_events_daily, on="date", how="left")

df_fe["event_attendance"] = df_fe["event_attendance"].fillna(0.0)
df_fe["event_names"] = df_fe["event_names"].fillna("none")
df_fe["event_category"] = df_fe["event_category"].fillna("none")
df_fe["event_flag"] = (df_fe["event_attendance"] > 0).astype(int)

# ======================================================
# 3. FEATURES TEMPORALES
# ======================================================

df_fe["dow"] = df_fe["date"].dt.weekday
df_fe["is_weekend"] = df_fe["dow"].isin([5, 6]).astype(int)

df_fe["dow_sin"] = np.sin(2*np.pi * df_fe["dow"] / 7)
df_fe["dow_cos"] = np.cos(2*np.pi * df_fe["dow"] / 7)

df_fe["month_sin"] = np.sin(2*np.pi * (df_fe["month"] - 1) / 12)
df_fe["month_cos"] = np.cos(2*np.pi * (df_fe["month"] - 1) / 12)

# ======================================================
# 4. LAGS DE VIAJES POR MUNICIPIO (1–7)
# ======================================================

df_fe = df_fe.sort_values(["municipio_origen_name", "date"])

for lag in range(1, 8):
    df_fe[f"viajes_lag{lag}"] = (
        df_fe.groupby("municipio_origen_name")["viajes"].shift(lag)
    )

df_fe["viajes_mean7"] = (
    df_fe.groupby("municipio_origen_name")["viajes"]
         .transform(lambda s: s.shift(1).rolling(7, min_periods=1).mean())
)

# ======================================================
# 5. LAGS DE TOTAL VIAJES DÍA (1–7) — CORRECTO
# ======================================================

daily_totals = df_fe.groupby("date")["viajes"].sum().rename("total_viajes_dia")
df_fe = df_fe.merge(daily_totals, on="date", how="left")

# Lags del total por fecha (expandido por merge)
for lag in range(1, 8):
    lag_series = daily_totals.shift(lag).rename(f"total_viajes_dia_lag{lag}")
    df_fe = df_fe.merge(lag_series, left_on="date", right_index=True, how="left")

# Mean 7 días retrasado
mean7 = (
    daily_totals.shift(1).rolling(7, min_periods=1).mean()
              .rename("total_viajes_dia_mean7")
)
df_fe = df_fe.merge(mean7, left_on="date", right_index=True, how="left")

# ======================================================
# 6. CAST DE VARIABLES CATEGÓRICAS
# ======================================================

df_fe["municipio_origen_name"] = df_fe["municipio_origen_name"].astype("category")
df_fe["origen"] = df_fe["origen"].astype("category")
df_fe["day_of_week"] = df_fe["day_of_week"].astype("category")
df_fe["event_category"] = df_fe["event_category"].astype("category")

# ======================================================
# 7. ELIMINAR FILAS SIN LAGS
# ======================================================

lag_cols = (
    [f"viajes_lag{i}" for i in range(1, 8)]
    + ["viajes_mean7"]
    + [f"total_viajes_dia_lag{i}" for i in range(1, 8)]
    + ["total_viajes_dia_mean7"]
)

df_model = df_fe.dropna(subset=lag_cols).copy()

# ======================================================
# 8. FEATURE SET FINAL
# ======================================================

target_col = "viajes"

feature_cols = [
    "municipio_origen_name",
    "origen",
    "day_of_week",

    "month", "dow", "is_weekend",
    "dow_sin","dow_cos","month_sin","month_cos",

    "tavg","tmin","tmax","prcp",

    "event_flag", "event_attendance", "event_category",
] + \
[ f"viajes_lag{i}" for i in range(1, 8) ] + \
[ "viajes_mean7" ] + \
[ f"total_viajes_dia_lag{i}" for i in range(1, 8) ] + \
[ "total_viajes_dia", "total_viajes_dia_mean7" ]

print("\n============================")
print("df_model shape:", df_model.shape)
print("Número de features:", len(feature_cols))
print("============================")

display(df_model[feature_cols + [target_col]].head(20))



df_model shape: (181692, 44)
Número de features: 34


,municipio_origen_name,origen,day_of_week,month,dow,is_weekend,dow_sin,dow_cos,month_sin,month_cos,tavg,tmin,tmax,prcp,event_flag,event_attendance,event_category,viajes_lag1,viajes_lag2,viajes_lag3,viajes_lag4,viajes_lag5,viajes_lag6,viajes_lag7,viajes_mean7,total_viajes_dia_lag1,total_viajes_dia_lag2,total_viajes_dia_lag3,total_viajes_dia_lag4,total_viajes_dia_lag5,total_viajes_dia_lag6,total_viajes_dia_lag7,total_viajes_dia,total_viajes_dia_mean7,viajes
28,Alella,Regional,Domingo,1,6,1,-0.781831,0.623490,0.0,1.0,12.7,9.1,15.8,0.0,0,0.0,none,313.0,2042.0,100.0,697.0,1217.0,228.0,1670.0,895.285714,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,341436,508896.857143,1303
29,Alella,Residente,Domingo,1,6,1,-0.781831,0.623490,0.0,1.0,12.7,9.1,15.8,0.0,0,0.0,none,1303.0,313.0,2042.0,100.0,697.0,1217.0,228.0,842.857143,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,341436,508896.857143,859
30,Alella,Nacional,Domingo,1,6,1,-0.781831,0.623490,0.0,1.0,12.7,9.1,15.8,0.0,0,0.0,none,859.0,1303.0,313.0,2042.0,100.0,697.0,1217.0,933.000000,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,341436,508896.857143,90
31,Alella,Internacional,Domingo,1,6,1,-0.781831,0.623490,0.0,1.0,12.7,9.1,15.8,0.0,0,0.0,none,90.0,859.0,1303.0,313.0,2042.0,100.0,697.0,772.000000,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,341436,508896.857143,175
32,Alella,Regional,Lunes,1,0,0,0.000000,1.000000,0.0,1.0,14.1,10.9,17.3,0.0,0,0.0,none,175.0,90.0,859.0,1303.0,313.0,2042.0,100.0,697.428571,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,653004,509723.428571,2717
33,Alella,Residente,Lunes,1,0,0,0.000000,1.000000,0.0,1.0,14.1,10.9,17.3,0.0,0,0.0,none,2717.0,175.0,90.0,859.0,1303.0,313.0,2042.0,1071.285714,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,653004,509723.428571,604
34,Alella,Nacional,Lunes,1,0,0,0.000000,1.000000,0.0,1.0,14.1,10.9,17.3,0.0,0,0.0,none,604.0,2717.0,175.0,90.0,859.0,1303.0,313.0,865.857143,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,653004,509723.428571,74
35,Alella,Internacional,Lunes,1,0,0,0.000000,1.000000,0.0,1.0,14.1,10.9,17.3,0.0,0,0.0,none,74.0,604.0,2717.0,175.0,90.0,859.0,1303.0,831.714286,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,653004,509723.428571,236
36,Alella,Nacional,Martes,1,1,0,0.781831,0.623490,0.0,1.0,9.9,6.1,15.2,0.0,0,0.0,none,236.0,74.0,604.0,2717.0,175.0,90.0,859.0,679.285714,653004.0,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,683244,522697.857143,76
37,Alella,Residente,Martes,1,1,0,0.781831,0.623490,0.0,1.0,9.9,6.1,15.2,0.0,0,0.0,none,76.0,236.0,74.0,604.0,2717.0,175.0,90.0,567.428571,653004.0,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,683244,522697.857143,560


In [42]:
display(df_model.head())

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes,tavg,tmin,tmax,prcp,event(y/n),name,attendance,date,event_attendance,event_names,event_category,event_flag,dow,is_weekend,dow_sin,dow_cos,month_sin,month_cos,viajes_lag1,viajes_lag2,viajes_lag3,viajes_lag4,viajes_lag5,viajes_lag6,viajes_lag7,viajes_mean7,total_viajes_dia,total_viajes_dia_lag1,total_viajes_dia_lag2,total_viajes_dia_lag3,total_viajes_dia_lag4,total_viajes_dia_lag5,total_viajes_dia_lag6,total_viajes_dia_lag7,total_viajes_dia_mean7
28,2023-01-08,Domingo,1,Regional,8003,Alella,8019,Barcelona,1303,12.7,9.1,15.8,0.0,n,NaN,NaN,2023-01-08,0.0,none,none,0,6,1,-0.781831,0.62349,0.0,1.0,313.0,2042.0,100.0,697.0,1217.0,228.0,1670.0,895.285714,341436,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,508896.857143
29,2023-01-08,Domingo,1,Residente,8003,Alella,8019,Barcelona,859,12.7,9.1,15.8,0.0,n,NaN,NaN,2023-01-08,0.0,none,none,0,6,1,-0.781831,0.62349,0.0,1.0,1303.0,313.0,2042.0,100.0,697.0,1217.0,228.0,842.857143,341436,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,508896.857143
30,2023-01-08,Domingo,1,Nacional,8003,Alella,8019,Barcelona,90,12.7,9.1,15.8,0.0,n,NaN,NaN,2023-01-08,0.0,none,none,0,6,1,-0.781831,0.62349,0.0,1.0,859.0,1303.0,313.0,2042.0,100.0,697.0,1217.0,933.000000,341436,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,508896.857143
31,2023-01-08,Domingo,1,Internacional,8003,Alella,8019,Barcelona,175,12.7,9.1,15.8,0.0,n,NaN,NaN,2023-01-08,0.0,none,none,0,6,1,-0.781831,0.62349,0.0,1.0,90.0,859.0,1303.0,313.0,2042.0,100.0,697.0,772.000000,341436,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,335650.0,508896.857143
32,2023-01-09,Lunes,1,Regional,8003,Alella,8019,Barcelona,2717,14.1,10.9,17.3,0.0,n,NaN,NaN,2023-01-09,0.0,none,none,0,0,0,0.000000,1.00000,0.0,1.0,175.0,90.0,859.0,1303.0,313.0,2042.0,100.0,697.428571,653004,341436.0,472945.0,391213.0,568603.0,628265.0,603419.0,562183.0,509723.428571


In [43]:
import os

# Ruta destino
output_path = "data/processed/df_model.csv"

# Crear carpeta si no existe
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Guardar CSV
df_model.to_csv(output_path, index=False)

print(f"✔ Archivo guardado en: {output_path}")
print(f"✔ Filas: {len(df_model)}, Columnas: {len(df_model.columns)}")


✔ Archivo guardado en: data/processed/df_model.csv
✔ Filas: 181692, Columnas: 44
